In [ ]:
# default_exp core

# pyshell

> A Shell object that allows the user to access every program in the `PATH` as an object method. 

In [ ]:
#hide
from nbdev.showdoc import *

## Imports

In [ ]:
#export
import subprocess
from fastcore.basics import *
from fastcore.all import *

## Command

In [ ]:
#export
class Command:
    """A shell command."""
    def __init__(self, name): self.name = name
    def __call__(self, *args, shell=True, decode='utf-8', out_error=False, ignore_error=False,
                 replace_underscore='-', capture=True, **kwargs):
        command = [self.name]
        if 'flags' in kwargs:
            flags = kwargs.pop('flags')
            for flag in flags:
                if len(flag)==1:
                    command.append('-' + flag)
                else:
                    command.append('--' + flag.replace('_', replace_underscore))
        for k in kwargs:
            if len(k)==1:
                command.append('-' + k)
            else:
                command.append('--' + k.replace('_', replace_underscore))
            command.append(str(kwargs[k]))
        command.extend([str(x) for x in args])
        if capture: st = {'stdout': subprocess.PIPE, 'stderr': subprocess.PIPE}
        else: st = {}
        process = subprocess.Popen(' '.join(command), shell=shell, **st)
        output, error = process.communicate()
        if error:
            error = error.decode("utf-8").strip('\n')
            if out_error:
                return error
            elif not ignore_error:
                raise Exception(error)
        if decode and output:
            output = output.decode("utf-8").strip('\n')
        return output

In [ ]:
ls = Command('ls')
test_eq(ls("*.ipynb"), '00_core.ipynb\nindex.ipynb')

## Shell

In [ ]:
#export
class Shell:
    def __getattr__(self, k, *args, **kwargs):
        if k[0]=='_': return super().__getattr__(k)
        com = Command(k)
        com.__doc__ = Command('man')(k, out_error=True)
        return com 
    def _dir(self): return L(L(self.echo('$PATH').split(':')).map(self.ls).map(Self.split('\n')).sum()).unique().filter()
    def __dir__(self): return custom_dir(self, self._dir())

In [ ]:
sh = Shell()

In [ ]:
test_eq(sh.ls("*.ipynb"), '00_core.ipynb\nindex.ipynb')

In [ ]:
test_eq(sh.ls('\.giti*', flags='a'), '.gitignore')

In [ ]:
test_eq(sh.ls('\.giti*', flags=['asdf'], out_error=True), "ls: unrecognized option '--asdf'\nTry 'ls --help' for more information.")

In [ ]:
test_eq(sh.ls('\.giti*', flags=['asdf'], out_error=False, ignore_error=True), '')

In [ ]:
test_eq(sh.ls('\.giti*', flags=['asdf'], ignore_error=True), '')

## Export

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
